# Build a smart assistant to help mental health counselors respond thoughtfully to patients. 

## The Mental Health Chatbot (Multi-Turn with LLM + Classifier) works in **two steps**:

1. **Understanding the situation**: When you describe a patient's issue, the system uses a machine learning model to figure out what kind of response might be most helpful—like giving advice, validating feelings, asking a follow-up question, or sharing some mental health information.

2. **Generating a helpful reply**: After the system decides what type of response is appropriate, it asks a language model (Flan-T5) to write a suggestion based on that need. For example, if the model thinks the user needs validation, it will ask the LLM to generate an empathetic and supportive response.



## This following python code does:
- Classifies user messages into response types (advice, validation, information, question)
- Uses a language model (Flan-T5) to generate counselor-like responses
- Maintains a limited conversation history
- Allows exporting conversation history to a JSON file

### Load Required Libraries

In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from transformers import pipeline

/Users/Pi/miniconda3/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load and Label Dataset 

In [2]:
df = pd.read_csv("dataset/Kaggle_Mental_Health_Conversations_train.csv")
df = df[['Context', 'Response']].dropna().copy()

keywords_to_labels = {
    'advice': ['try', 'should', 'suggest', 'recommend'],
    'validation': ['understand', 'feel', 'valid', 'normal'],
    'information': ['cause', 'often', 'disorder', 'symptom'],
    'question': ['how', 'what', 'why', 'have you']
}

def auto_label_response(response):
    response = response.lower()
    for label, keywords in keywords_to_labels.items():
        if any(word in response for word in keywords):
            return label
    return 'information'

df['response_type'] = df['Response'].apply(auto_label_response)



### Train on Combined Context + Response

In [3]:
df['combined_text'] = df['Context'] + " " + df['Response']

le = LabelEncoder()
y = le.fit_transform(df['response_type'])

vectorizer = TfidfVectorizer(max_features=2000, ngram_range=(1, 2))
X = vectorizer.fit_transform(df['combined_text'])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

xgb_model = XGBClassifier(
    objective='multi:softmax',
    num_class=len(le.classes_),
    eval_metric='mlogloss',
    use_label_encoder=False,
    max_depth=6,
    learning_rate=0.1,
    n_estimators=100
)
xgb_model.fit(X_train, y_train)

/Users/Pi/miniconda3/envs/myenv/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [22:52:39] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None, num_class=4, ...)

### Load LLM (Flan-T5)

In [4]:
print("Loading Flan-T5 model... (this may take a few seconds)")
llm = pipeline("text2text-generation", model="google/flan-t5-base")

Loading Flan-T5 model... (this may take a few seconds)


Device set to use mps:0


### Prediction + Prompt Functions

In [5]:
def predict_response_type(user_input):
    combined = user_input + " placeholder_response"
    vec = vectorizer.transform([combined])
    prediction = xgb_model.predict(vec)[0]
    predicted_class = le.inverse_transform([prediction])[0]
    confidence = np.max(xgb_model.predict_proba(vec))
    return predicted_class, confidence

def prompt_templates(user_input, response_type):
    templates = {
        "advice": f"A student said: \"{user_input}\". What practical advice should a mental health counselor offer?",
        "validation": f"A student said: \"{user_input}\". Respond with an emotionally supportive message that shows empathy and validates their feelings.",
        "information": f"A student said: \"{user_input}\". Explain what might be happening emotionally from a counselor's perspective.",
        "question": f"A student said: \"{user_input}\". What are 1-2 thoughtful follow-up questions a counselor might ask?"
    }
    return templates.get(response_type, templates["information"])

def generate_llm_response(user_input, response_type):
    prompt = prompt_templates(user_input, response_type)
    result = llm(prompt, max_length=150, do_sample=True, temperature=0.7, top_p=0.9)
    return result[0]["generated_text"].strip()

### Conversation Memory + Exporting

In [6]:
MAX_MEMORY_TURNS = 6
history = []

def trim_memory(history, max_turns=MAX_MEMORY_TURNS):
    return history[-max_turns:]

def save_conversation(history, filename="chat_history.json"):
    with open(filename, "w") as f:
        json.dump(history, f, indent=2)
    print(f"\nConversation saved to {filename}")

### Intro + Chat

In [7]:
def show_intro():
    print("\n--- Multi-Turn Mental Health Chatbot ---")
    print("This assistant simulates a counselor's conversation using AI.")
    print("- Type something your patient/student might say")
    print("- Type 'save' to export the conversation")
    print("- Type 'exit' to quit\n")

    print("Example:")
    print("User: I feel like I’ll mess up my big presentation tomorrow.")
    print("Counselor: It’s completely normal to feel nervous before a big event...\n")

show_intro()


--- Multi-Turn Mental Health Chatbot ---
This assistant simulates a counselor's conversation using AI.
- Type something your patient/student might say
- Type 'save' to export the conversation
- Type 'exit' to quit

Example:
User: I feel like I’ll mess up my big presentation tomorrow.
Counselor: It’s completely normal to feel nervous before a big event...



### Chat Loop

In [8]:
while True:
    user_input = input("User: ").strip()

    if user_input.lower() == "exit":
        print("Goodbye")
        break
    elif user_input.lower() == "save":
        save_conversation(history)
        continue

    predicted_type, confidence = predict_response_type(user_input)
    print(f"(Predicted: {predicted_type}, Confidence: {confidence:.1%})")

    llm_reply = generate_llm_response(user_input, predicted_type)

    history.append({"role": "user", "content": user_input})
    history.append({"role": "assistant", "content": llm_reply})
    history = trim_memory(history)

    print("Counselor:", llm_reply)

User:  i'm nervous


(Predicted: information, Confidence: 85.5%)
Counselor: The student might be feeling anxious or uncertain about the situation.


User:  exit


Goodbye
